In [ ]:
#!pip3 install pandas
#!pip3 install scikit-learn
#!pip3 install scikeras[tensorflow]



In [1]:
# import libraries
#from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import Flatten, Conv1D, MaxPooling1D, LSTM
from keras.activations import relu, elu, linear, softmax
from keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam, Nadam
from keras.losses import mean_squared_error, categorical_crossentropy, logcosh
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
import time

from tensorflow.keras.optimizers import Adam, Nadam, SGD

/scratch/local/u6031121/300230/ipykernel_2670531/3226968778.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-01 02:42:15.107052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 02:42:15.107114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 02:42:15.247302: E external/local_xla/xla/stream_executor/cud

In [3]:
import keras_tuner
from keras_tuner import RandomSearch

In [2]:
# Check available devices
devices = tf.config.list_physical_devices('GPU')

if len(devices) > 0:
    print("GPU is available")
    for device in devices:
        print("Device name:", device.name)
else:
    print("No GPU is available")

GPU is available
Device name: /physical_device:GPU:0
Device name: /physical_device:GPU:1


In [21]:
# # processig data
# def data_preprocess(dataset_trn,target_trn): 
#     # Scaled data
#     min_max_scaler = MinMaxScaler(feature_range = (0,1))
#     np_scaled = min_max_scaler.fit_transform(dataset_trn)
#     X = pd.DataFrame(np_scaled)
    
#     target_edit = pd.Series(target_trn).values
#     target_edit = target_edit.reshape(-1,1)
#     np_scaled = min_max_scaler.fit_transform(target_edit)
#     Y = pd.DataFrame(np_scaled)
#     y_train = Y
#     # Split data
#     #from sklearn.model_selection import train_test_split
#     #X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
      
#     scaler = StandardScaler()
#     # Fit only to the training data
#     scaler.fit(X)
        
#     X_train = scaler.transform(X)
#     #X_test = scaler.transform(X_test)
    
#     X_train = pd.DataFrame(X_train)
#     #X_val = pd.DataFrame(X_test)
#     #y_val = y_test
#     return X_train, y_train

In [22]:
# # processig data
# def data_preprocess_tst(dataset_tst,target_tst): 
#     # Scaled data
#     min_max_scaler = MinMaxScaler(feature_range = (0,1))
#     np_scaled = min_max_scaler.fit_transform(dataset_tst)
#     X_tst = pd.DataFrame(np_scaled)
    
#     target_edit = pd.Series(target_tst).values
#     target_edit = target_edit.reshape(-1,1)
#     np_scaled = min_max_scaler.fit_transform(target_edit)
#     Y_tst = pd.DataFrame(np_scaled)
#     y_test = Y_tst
#     # Split data
#    # from sklearn.model_selection import train_test_split
#    # X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
      
#     scaler = StandardScaler()
#     # Fit only to the training data
#     scaler.fit(X_tst)
        
#     X_test = scaler.transform(X_tst)
#     X_test = pd.DataFrame(X_test)
    
#     return X_test, y_test

In [4]:
# processig data
def data_preprocess(dataset,target): 
    # Scaled data
    min_max_scaler = MinMaxScaler(feature_range = (0,1))
    np_scaled = min_max_scaler.fit_transform(dataset)
    X = pd.DataFrame(np_scaled)
    
    target_edit = pd.Series(target).values
    target_edit = target_edit.reshape(-1,1)
    np_scaled = min_max_scaler.fit_transform(target_edit)
    Y = pd.DataFrame(np_scaled)
    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
      
    scaler = StandardScaler()
    # Fit only to the training data
    scaler.fit(X_train)
        
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    return X_train, y_train, X_test, y_test

In [18]:
def cal_correlation(pred, y_test, target):
    pred = pred.reshape((-1,1))
    target_edit = pd.Series(target).values
    target_edit = target_edit.reshape(-1,1)
    min_max_scaler = MinMaxScaler(feature_range = (0,1))
    np_scaled = min_max_scaler.fit_transform(target_edit)
    target_pred = min_max_scaler.inverse_transform(pred)
    target_orig = min_max_scaler.inverse_transform(y_test)
    target_orig = target_orig[:,0]
    target_orig = pd.Series(target_orig)
    target_pred = target_pred[:,0]
    target_pred = pd.Series(target_pred)
    cor1 = target_orig.corr(target_pred, method='pearson')
    return cor1

In [24]:
X_train.shape

(396, 9999)

In [6]:
def build_model(hp):
    model = keras.Sequential()    
    for i in range(1, hp.Int('num_layers', 2, 20)):
        model.add(keras.layers.Dense(units=hp.Int('units_'+str(i),
                                           min_value=32,
                                           max_value=512, step=32),
                              activation='relu', input_dim=9999))
    model.add(keras.layers.Dense(1, activation="linear"))
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mean_absolute_error",
        metrics=["mean_absolute_error"])
    
    return model


In [7]:
tuner = keras_tuner.Hyperband(
    build_model,
    objective=keras_tuner.Objective('val_mean_absolute_error','min'),
    factor=3,
    hyperband_iterations=5,
    max_epochs=20,
    overwrite=True,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/rough/',
    max_consecutive_failed_trials=3,
    project_name='Feb29_mlp')

2024-03-01 02:43:27.753696: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22460 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:27:00.0, compute capability: 8.6
2024-03-01 02:43:27.754677: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22460 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:28:00.0, compute capability: 8.6


In [9]:
# tuner = keras_tuner.BayesianOptimization(
#     build_model,
#     objective=keras_tuner.Objective('val_mean_absolute_error','min'),
#     max_trials=50,
#     executions_per_trial=4,
#     overwrite=True,
#     #distribution_strategy=tf.distribute.MirroredStrategy(),
#     directory='/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/rough/',
#     max_consecutive_failed_trials=6,
#     project_name='Feb29_mlp')

In [ ]:
# Define the search space including the number of epochs
#tuner.search_space.update({'epochs': hp.Int('epochs', min_value=10, max_value=100, step=10)})


In [8]:
data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/rough/Trn358_10kDL.csv', header=0)
#data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/NAM_dat.csv', header=0)
dataset=data.iloc[:,1:10000]
target=data.iloc[:, 10003]
#preprocess data
X_train, y_train, X_test, y_test = data_preprocess(dataset, target)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(X_train, y_train, epochs=30, validation_split=0.15, callbacks=[early_stopping] )


Trial 150 Complete [00h 00m 02s]
val_mean_absolute_error: 0.23707647621631622

Best val_mean_absolute_error So Far: 0.07773027569055557
Total elapsed time: 00h 09m 33s


In [9]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [10]:
best_hps

In [42]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_19')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384.



In [56]:
data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/rough/Trn358_10kDL.csv', header=0)
#data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/NAM_dat.csv', header=0)
dataset=data.iloc[:,1:10000]
target=data.iloc[:, 10003]
#preprocess data
X_train, y_train, X_test, y_test = data_preprocess(dataset, target)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#tuner.search(X_train, y_train, epochs=30, validation_split=0.15, callbacks=[early_stopping] )


In [57]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_mean_absolute_error']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
8/8 [==============================] - 3s 32ms/step - loss: 0.4987 - mean_absolute_error: 0.4987 - val_loss: 0.2061 - val_mean_absolute_error: 0.2061
Epoch 2/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1242 - mean_absolute_error: 0.1242 - val_loss: 0.1068 - val_mean_absolute_error: 0.1068
Epoch 3/50
8/8 [==============================] - 0s 9ms/step - loss: 0.0925 - mean_absolute_error: 0.0925 - val_loss: 0.1236 - val_mean_absolute_error: 0.1236
Epoch 4/50
8/8 [==============================] - 0s 9ms/step - loss: 0.1083 - mean_absolute_error: 0.1083 - val_loss: 0.1121 - val_mean_absolute_error: 0.1121
Epoch 5/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0963 - mean_absolute_error: 0.0963 - val_loss: 0.1114 - val_mean_absolute_error: 0.1114
Epoch 6/50
8/8 [==============================] - 0s 8ms/step - loss: 0.0930 - mean_absolute_error: 0.0930 - val_loss: 0.1212 - val_mean_absolute_error: 0.1212
Epoch 7/50
8/8 [=======================

In [58]:
hypermodel = tuner.hypermodel.build(best_hps)

In [59]:
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/38
8/8 [==============================] - 3s 33ms/step - loss: 0.4032 - mean_absolute_error: 0.4032 - val_loss: 1.6474 - val_mean_absolute_error: 1.6474
Epoch 2/38
8/8 [==============================] - 0s 8ms/step - loss: 0.3844 - mean_absolute_error: 0.3844 - val_loss: 0.1078 - val_mean_absolute_error: 0.1078
Epoch 3/38
8/8 [==============================] - 0s 9ms/step - loss: 0.0938 - mean_absolute_error: 0.0938 - val_loss: 0.1076 - val_mean_absolute_error: 0.1076
Epoch 4/38
8/8 [==============================] - 0s 8ms/step - loss: 0.0962 - mean_absolute_error: 0.0962 - val_loss: 0.1108 - val_mean_absolute_error: 0.1108
Epoch 5/38
8/8 [==============================] - 0s 9ms/step - loss: 0.0918 - mean_absolute_error: 0.0918 - val_loss: 0.1071 - val_mean_absolute_error: 0.1071
Epoch 6/38
8/8 [==============================] - 0s 9ms/step - loss: 0.0919 - mean_absolute_error: 0.0919 - val_loss: 0.1098 - val_mean_absolute_error: 0.1098
Epoch 7/38
8/8 [=======================

In [60]:
y_pred = hypermodel.predict(X_test)

3/3 [==============================] - 0s 2ms/step


In [61]:
cor = cal_correlation(y_pred, y_test, target)
cor

-0.08240123113220507

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
tuner.results_summary()

In [24]:
best_trial = tuner.oracle.get_best_trials(1)[0]

In [25]:
best_trial.hyperparameters.values

{'num_layers': 17,
 'units_1': 320,
 'lr': 0.004371994156211472,
 'units_2': 288,
 'units_3': 128,
 'units_4': 192,
 'units_5': 96,
 'units_6': 128,
 'units_7': 448,
 'units_8': 64,
 'units_9': 128,
 'units_10': 288,
 'units_11': 256,
 'units_12': 416,
 'units_13': 128,
 'units_14': 192,
 'units_15': 256,
 'units_16': 384,
 'units_17': 288,
 'units_18': 128,
 'units_19': 384,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0142'}

In [ ]:
model = final_model_MLP()

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split=0.2)

In [ ]:
loss, mae = model.evaluate(X_test, y_test)

In [ ]:
best_trial = tuner.oracle.get_best_trials(1)[0]

# Get the best hyperparameters
best_hyperparameters = best_trial.hyperparameters.values

# Build the model using the best hyperparameters
best_model = tuner.hypermodel.build(best_hyperparameters)

# Train the model on the full training data
best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Make predictions on the test data
test_predictions = best_model.predict(X_test)

# Evaluate the model on the test data if ground truth labels are available
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

In [ ]:
best_trial = tuner.oracle.get_best_trials(1)[0]
best_hyperparameters = best_trial.hyperparameters.values
num_layers = best_hyperparameters['num_layers']
units_list = [best_hyperparameters[f'units_{i}'] for i in range(num_layers)]
learningrate = best_hyperparameters['lr']
#best_model = build_model(best_hyperparameters)
best_hyperparameters

In [26]:
# Get predicted values from the best model
y_pred = model.predict(X_test)

# Assuming `y_pred` contains your predicted values
#print("Predicted values:", y_pred)
# Assuming `y_test` contains the true labels for the test data
# Assuming `y_pred` contains the predicted values obtained from the best-fit model

# Flatten the arrays if needed
y_test_flat = y_test.values
y_test_flat = y_test_flat.flatten()
y_pred_flat = y_pred.flatten()

# Calculate the correlation coefficient
correlation = np.corrcoef(y_test_flat, y_pred_flat)[0, 1]

# Print the correlation coefficient
print("Correlation coefficient:", correlation)

3/3 [==============================] - 0s 2ms/step
Correlation coefficient: 0.06118766863633908


In [27]:
cor = cal_correlation(y_pred, y_test, target)
cor

0.06118759439915612

In [28]:
# calculate average of items in a list
def Average(lst): 
    return sum(lst) / len(lst)

In [ ]:
def main():
    #start time
    start = time.time()
    
    #data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/NAM_dat.csv', header=0)
    dataset=data.iloc[:,19:]
    target=data.iloc[:, 4]
    
    #preprocess data
    X_train, y_train, X_test, y_test = data_preprocess(dataset, target)
    
    #store all cor and accuracy values for mlp
    cor_mlp = []
    acc_mlp = []
    #store all cor and accuracy values for cnn
    cor_cnn = []
    acc_cnn = []
    
    #200 iterations
    for i in range(10):
        # Multilayer perceptron
        print('MLP:')
        scores_mlp, pred_mlp = multi_layer_perceptron(X_train, y_train, X_test, y_test)
        cor_m = cal_correlation(pred_mlp, y_test, target)
        print('Correlation for MLP ',cor_m)
        cor_mlp.append(cor_m)
        acc_mlp.append(scores_mlp)
    
        # CNN
        #print('CNN:')
       # scores_cnn, pred_cnn = cnn(X_train, y_train, X_test, y_test)
       # cor_c = cal_correlation(pred_cnn, y_test, target)
       # print('Correlation for CNN ',cor_c)
       # cor_cnn.append(cor_c)
       # acc_cnn.append(scores_mlp)
    
    #average values
    print("Average accuracy for MLP: ", Average(acc_mlp))
    print("Average correlation for MLP: ", Average(cor_mlp))
   # print("Average accuracy for CNN: ", Average(acc_cnn))
   # print("Average correlation for CNN: ", Average(cor_cnn))
    
    #end time
    end = time.time()
    print("Time elapsed: ",end - start)
    
main()

In [ ]:
best_trials = tuner.oracle.get_best_trials(1)
best_trial = best_trials[0]  # Assuming you want information about the top best trial

# Accessing information about the best trial
best_hyperparameters = best_trial.hyperparameters.values
best_model_dir = best_trial.trial_id  # Directory where the best model checkpoints are saved
best_score = best_trial.score 

In [ ]:
#model.add(keras.layers.Flatten())
#     model.add(
#         keras.layers.Dense(
#             # Tune number of units.
#             units=hp.Int("units", min_value=32, max_value=512, step=32),
#             # Tune the activation function to use.
#             activation=hp.Choice("activation", ["relu","tanh"]),
#         )
#     )
#     # Tune whether to use dropout.
#     if hp.Boolean("dropout"):
#         model.add(keras.layers.Dropout(rate=0.25))